In [3]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import ast, re
from time import sleep
from bs4 import BeautifulSoup
external_data_folder = 'external_data/'
data_folder = ''
api = ''
import warnings
warnings.filterwarnings('ignore')

### Extract terms from text data here

In [13]:
text_df = pd.read_csv(data_folder+'data/drug_ade_data_raw.csv')
text_df['section_title'] = text_df['drug_text'].apply(lambda x: BeautifulSoup(x, 'html.parser').find('summary').text)
text_df['section_content'] = text_df.apply(lambda x: BeautifulSoup(x.drug_text, 'html.parser').text.replace(x.section_title, '').strip().lower(),
                                           axis = 1)
text_df.head(1)

,product_id,drug_text,section_title,section_content
0,1589,"<details><summary data-evt=""smpcSectionOpen"" i...",4.8 Undesirable effects,hypersensitivity reactions may occasionally oc...


In [19]:
unique_text = text_df[['section_content']].drop_duplicates()
print(unique_text.shape[0])
unique_text.head(1)

7699


,section_content
0,hypersensitivity reactions may occasionally oc...


In [20]:
#read in the meddra map file
f = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/'
meddra_df = pd.read_csv(f+'external_data/umls_meddra_en.csv')
meddra_df['STR'] = meddra_df.STR.apply(lambda x: x.lower())
meddra_df['len'] = meddra_df.STR.apply(lambda x: len(x))
print(meddra_df.shape[0])
meddra_df = meddra_df[(meddra_df['len'] >= 5)]
meddra_df = meddra_df[(meddra_df.TTY.isin(['PT', 'LLT']))]
print(meddra_df.shape[0])
meddra_dict = dict(zip(meddra_df.STR, meddra_df.SDUI))
meddra_names = meddra_df.STR.tolist()
meddra_df.head(1)

116807
100897


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,len
0,C0000727,ENG,P,L0000727,VCW,S0584932,N,A0639292,NaN,NaN,10000647,MDR,PT,10000647,acute abdomen,3,N,256.0,13


In [21]:
app_data = []
for ade_text in tqdm(unique_text.section_content.tolist()):
  if str(ade_text) != 'nan':
    ade_text = ade_text.lower()
    meddra_found_terms = []
    #iterate through list of meddra concepts, if found - add to list
    for concept_name in meddra_names:
      if ade_text.find(concept_name) == -1:
        continue
      else:
        i = ade_text.index(concept_name)
        meddra_found_terms.append([i, concept_name, meddra_dict[concept_name]])
    app_data.append([ade_text, meddra_found_terms])

100%|██████████| 7699/7699 [43:56<00:00,  2.92it/s]


In [22]:
app_data_df = pd.DataFrame(app_data, columns = ['string', 'list'])
app_data_df = app_data_df.explode('list')
app_data_df['index'] = app_data_df['list'].apply(lambda x: x[0] if str(x) != 'nan' else None)
app_data_df['found_term'] = app_data_df['list'].apply(lambda x: x[1] if str(x) != 'nan' else None)
app_data_df['meddra_id'] = app_data_df['list'].apply(lambda x: x[2] if str(x) != 'nan' else None)
app_data_df = app_data_df.drop(['list'], axis = 1).drop_duplicates()
app_data_df = app_data_df[app_data_df.found_term.notna()]
app_data_df.to_csv(data_folder+'data/drug_ade_text_matched_terms.csv', index=False)
app_data_df.head(1)

,string,index,found_term,meddra_id
0,hypersensitivity reactions may occasionally oc...,0.0,hypersensitivity,10020751.0


### prepare onsides input

In [3]:
text_df = pd.read_csv(data_folder+'data/drug_ade_data_raw.csv')
text_df['section_title'] = text_df['drug_text'].apply(lambda x: BeautifulSoup(x, 'html.parser').find('summary').text)
text_df['section_content'] = text_df.apply(lambda x: BeautifulSoup(x.drug_text, 'html.parser').text.replace(x.section_title, '').strip().lower(),
                                           axis = 1)
text_df.head(1)

,product_id,drug_text,section_title,section_content
0,1589,"<details><summary data-evt=""smpcSectionOpen"" i...",4.8 Undesirable effects,hypersensitivity reactions may occasionally oc...


In [4]:
text_df.section_content.tolist()[0]

'hypersensitivity reactions may occasionally occur in certain individuals following repeated application.\n reporting of suspected adverse reactions\n reporting suspected adverse reactions after authorisation of the medicinal product is important.  it allows continued monitoring of the benefit/risk balance of the medicinal product.  healthcare professionals are asked to report any suspected adverse reactions via the yellow card scheme at www.mhra.gov.uk/yellowcard.'

In [5]:
app_data_df = pd.read_csv(data_folder+'data/drug_ade_text_matched_terms.csv')
print(app_data_df.shape)
app_data_df = app_data_df.drop_duplicates()
print(app_data_df.shape)
app_data_df.head(1)

(761956, 4)
(761956, 4)


,string,index,found_term,meddra_id
0,hypersensitivity reactions may occasionally oc...,0.0,hypersensitivity,10020751.0


In [7]:
building_strings = []
#ade_text_table_dict = dict(zip(text_df.product_id, text_df.section_content))
for i, row in tqdm(app_data_df.iterrows()):
  if str(row['index']) != 'nan':
    term, start_pos, ar_text = row['found_term'], int(row['index']), row['string']
    #default settings
    nwords, prop_before = 125, 0.125
    #pull the full text
    #ar_text = ade_text_table_dict[label_id]

    term_nwords = len(term)
    size_before = max(int((nwords-2*term_nwords)*prop_before), 1)
    size_after = max(int((nwords-2*term_nwords)*(1-prop_before)), 1)

    before_text = ar_text[:start_pos]
    after_text = ar_text[(start_pos+term_nwords):]

    before_parts = before_text.split()[-1*size_before:]
    after_parts = after_text.split()[:size_after]

    li = [term]
    li.extend(before_parts)
    li.append('EVENT')
    li.extend(after_parts)
    example_string = ' '.join(li)
    building_strings.append(example_string)
  else:
    building_strings.append(None)

761956it [01:40, 7616.71it/s]


In [11]:
app_data_df['section_content'] = app_data_df['string']
app_data_df['onsides_string'] = building_strings
exact_terms_df = text_df.merge(app_data_df, on = 'section_content', how = 'left').drop(columns = ['drug_text', 'section_title', 'string'])
exact_terms_df = exact_terms_df.rename(columns = {'index':'location', 'onsides_string':'string'})
exact_terms_df.head()

,product_id,section_content,location,found_term,meddra_id,string
0,1589,hypersensitivity reactions may occasionally oc...,0.0,hypersensitivity,10020751.0,hypersensitivity EVENT reactions may occasiona...
1,1589,hypersensitivity reactions may occasionally oc...,0.0,hypersensitivity reaction,10020751.0,hypersensitivity reaction EVENT s may occasion...
2,1589,hypersensitivity reactions may occasionally oc...,121.0,spect,10040722.0,spect hypersensitivity reactions may occasiona...
3,1589,hypersensitivity reactions may occasionally oc...,129.0,adverse reaction,10067484.0,adverse reaction occasionally occur in certain...
4,4223,the list of the following adverse effects rela...,921.0,angioedema,10002424.0,"angioedema oral discomfortab a2,4-dichlorobenz..."


In [12]:
exact_terms_df.to_csv(data_folder+'data/ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv', index=False)

---
### format dataframe for onsides model

In [18]:
#required columns : section, drug, label_id, set_id, spl_version, pt_meddra_id, pt_meddra_term
exact_terms_df = pd.read_csv(data_folder+'data/ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv')
exact_terms_df['section'] = 'AR'
exact_terms_df['set_id'] = exact_terms_df['product_id']
exact_terms_df['label_id'] = exact_terms_df['product_id']
print(exact_terms_df.shape)
drug_df = pd.read_csv(data_folder+'data/final_UK/drug_data.csv').rename(columns={'ingredient_name':'drug'})
drug_df['label_id'] = drug_df['product_id'].apply(lambda x: int(x.split('/')[-2]))
drug_df = drug_df[['label_id', 'drug']].drop_duplicates().groupby('label_id')['drug'].apply(set).reset_index()
drug_df['drug'] = drug_df['drug'].apply(lambda x: str(x)[1:-1].replace("'",''))
exact_terms_df = exact_terms_df.merge(drug_df[['label_id', 'drug']].drop_duplicates(), on = 'label_id', how = 'left')
print(exact_terms_df.shape)
folder = 'data/'
llt_pt = pd.read_csv(folder+'meddra_llt_pt_map.txt', delimiter = '|')
llt_pt_id_dict = dict(zip(llt_pt.llt_concept_code, llt_pt.pt_concept_code))
llt_pt_term_dict = dict(zip(llt_pt.llt_concept_code, llt_pt.pt_concept_name))
exact_terms_df['pt_meddra_id'] = exact_terms_df.meddra_id.apply(lambda x: llt_pt_id_dict[x] if x in llt_pt_id_dict.keys() else None)
exact_terms_df['pt_meddra_term'] =  exact_terms_df.meddra_id.apply(lambda x: llt_pt_term_dict[x] if x in llt_pt_term_dict.keys() else None)
print(exact_terms_df.shape)
exact_terms_df = exact_terms_df[exact_terms_df.string.notna()]
exact_terms_df['spl_version'] = None
print(exact_terms_df.shape)
exact_terms_df.to_csv(data_folder+'data/ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv', index=False)
exact_terms_df.head(1)

(924331, 9)
(924331, 10)
(924331, 12)
(924318, 13)


,product_id,section_content,location,found_term,meddra_id,string,section,set_id,label_id,drug,pt_meddra_id,pt_meddra_term,spl_version
0,1589,hypersensitivity reactions may occasionally oc...,0.0,hypersensitivity,10020751.0,hypersensitivity EVENT reactions may occasiona...,AR,1589,1589,"cetrimide, allantoin, 2,4-dichlorobenzyl alcohol",10020751.0,Hypersensitivity,None


### run onsides model

In [2]:
#prep onsides materials
folder = 'onsides'
ar_model = folder + '/bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth'
bw_model = folder + '/bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth'
import os
os.chdir('onsides-2.0.0/')
!python3 -m pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.3/292.3 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 25.1 MB/

In [20]:
#use onsides model
f = data_folder+'data/ade/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0120.csv'
!python3 src/predict.py --model $ar_model --examples $f

2024-01-28 19:44:04.955610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 19:44:04.955688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 19:44:05.068008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 19:44:05.320085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 19:44:07.047749: W tensorflow/comp

In [4]:
#build tables
r = data_folder+'data/serious/bestepoch-bydrug-PMB-sentences-rx_ref14-AR-125-all_222_24_25_2.5e-05_256_32.csv.gz'
!python3 src/create_onsides_datafiles.py --release v2.0.0-AR --results $r --examples $f

usage: create_onsides_datafiles.py [-h] --results RESULTS --examples EXAMPLES --release RELEASE
create_onsides_datafiles.py: error: argument --results: expected one argument


In [ ]:
res = data_folder+'data/bestepoch-bydrug-PMB-sentences-rx_ref14-AR-125-all_222_24_25_2.5e-05_256_32.csv.gz'
ex = data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv'
#threshold = 0.463
res = pd.read_csv(res, header=None, names=['Pred0', 'Pred1'])
ex = pd.read_csv(ex)
df = pd.concat([ex, res], axis=1)
print(df.shape[0])
df = df[df.Pred0 > threshold]
print(df.shape[0])
df.head()

854253
625380


,label_id,found_term,meddra_id,location,string,section,set_id,drug,spl_version,pt_meddra_id,pt_meddra_term,Pred0,Pred1
0,1589,hypersensitivity,10020751,0,hypersensitivity EVENT ypersensitivity reactio...,AR,1589,NaN,NaN,10020751.0,Hypersensitivity,1.179923,0.0
1,1589,hypersensitivity reaction,10020751,0,hypersensitivity reaction EVENT persensitivity...,AR,1589,NaN,NaN,10020751.0,Hypersensitivity,1.194894,0.0
2,1589,hyper,10037211,0,hyper EVENT ypersensitivity reactions may occa...,AR,1589,NaN,NaN,10037211.0,Psychomotor hyperactivity,1.665707,0.0
3,4223,angioedema,10002424,729,angioedema presented in order of decreasing se...,AR,4223,NaN,NaN,10002424.0,Angioedema,1.494658,0.0
4,4223,edema,10030095,734,"edema in order of decreasing seriousness.a2,4-...",AR,4223,NaN,NaN,10030095.0,Oedema,2.323206,0.0


In [ ]:
df.to_csv(data_folder+'data/ade_text_table_onsides_pred_v0924.csv', index=False)